In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
N, D1, D2 = train_data.data.shape
#_, Nc = train_data.targets.shape
D = D1 * D2 
Nc = 10 # #classes
N, D1, D2, D, Nc
class MyModel(nn.Module):
    def __init__(self, D:int, Nc:int):
        super().__init__()
        self.flattern = nn.Flatten()
        self.net = nn.Sequential(
            nn.Linear(D, 256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128, Nc),
        )
    def forward(self,x):
        x = self.flattern(x)
        logits = self.net(x)
        return logits
    
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MyModel(D, Nc).to(device)
print(model)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

Epochs = 2
loss_d = {'epoch': [], 'val':[]}
for i in range(Epochs):
    print(f'Epoch: {i}')
    size = len(train_dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss_v, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    loss_d['epoch'].append(i)
    loss_d['val'].append(loss_v)
    
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size 
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

plt.plot(loss_d['epoch'], loss_d['val'])